### Threads
There are two primary ways to create a thread. 1) Extend the `Thread` class and override the run method.

In [ ]:
public class Worker extends Thread{
    @Override
    public void run(){
        System.out.println("Worker thread running");
    }
}

Worker worker = new Worker();

2) The preferred way is to pass a `Runnable` instance to Thread constructor

In [ ]:
Runnable runnable = new Runnable{
    public void run(){
        System.out.println("Worker thread running");
    }
}

Thread worker = new Thread(runnable);

**Starting a thread:** use the `start` method of the Thread class. The `isAlive` method can tell if the thread is currently running or not. isAlive returns false if the thread has not been started or the thread has been terminated.  

**Stopping a thread:** is not as straight forward as starting it. The only way to terminate a thread is to arrange for its run method to complete. The thread class has a `stop` method, but it has been deprecated. It has been deprecated because stopping a thread releases all the locks it may have. If the thread was inside a critical section when stopped, it would lead to corrupted state. Other threads which access that critical section might be working with corrupted state.

Thread throwing an unchecked exception can also terminate the thread. Two approaches for stopping a thread: 
1) setting a flag

In [ ]:
public class Worker extends Thread{
    private volatile boolean done = false;
    
    @Override
    public void run(){
        while(!done){
            // Do something
        }
    }
    
    // Other methods
}

An external code can stop this thread by setting done as true. But the thread will only stop when the execution reaches the top of the loop (and not immediately).  
2) Using interrupts: we can call `interrupt` method of any thread. It has two effects. First, it sets internal interrupted flag of the thread to true. Second, it causes any blocking method like `sleep`, `join`, `wait` to stop blocking and throw `InterruptedException`.

In [ ]:
public class Worker extends Thread{
    
    @Override
    public void run(){
        while(!isInterrupted()){
            // Do something
        }
    }
    
    // Other methods
}

Worker worker = new Worker();
worker.start();
worker.interrupt();

**Pausing a thread:** the Thread class has `suspend` and `resume` methods which have been deprecated (due to possibility of deadlock). It is still possible for a thread to suspend its own execution for a specified amount of time using `sleep` method. The least amount of time a thread can sleep on most Java implementations is 20 to 50 ms.

In [6]:
public class DelayedRandomGenerator extends Thread {
    public void run() {
        Random random = new Random();
        while (!isInterrupted()) {
            System.out.print(random.nextInt() + "\t");
            try {
                sleep(200);
            } catch (InterruptedException e) {
                break;
            }
        }
    }
}

DelayedRandomGenerator generator = new DelayedRandomGenerator();
generator.start();

try {
    Thread.sleep(1000);
} catch (InterruptedException e) {
    System.exit(0);
}        

generator.interrupt();

1152488192	2140810343	1285862342	1650368227	-1184878083	1039227028	278517272	-1543200418	401414008	-531716027	

Unlike primitives like int, float and double are not read from memory in one atomic step. Rather, 32 bits are read at a time. So it may be possible to read 32 bits of fresh value and 32 bits of stale value.

### Thread Notification
The `wait` and `notify` semantics allows threads to communicate among each other. Wait (and its overloaded variants) forces a thread to wait and notify signals a waiting thread that it can continue execution. Both methods must be called from within a synchronized block. Synchronized block because of the inherent race condition:
1. Thread A tests a condition and discovers it must wait.
2. Thread B sets the condition and calls notify() to inform A to resume execution. Because A is not yet waiting, nothing happens.
3. Thread A waits, by calling wait().
4. Because of the prior notify() call, A waits indefinitely.

The thread that calls wait (the waiting thread) and the thread that calls notify (the notification thread) must compete for the same lock. Either thread must call wait or notify via the same object on which they enter their synchronized contexts because wait tightly integrates with the lock. Prior to waiting, a thread executing wait releases the lock, which allows the notification thread to enter its synchronized context to set the condition and notify the waiting thread. Notify on the other hand doesn't release any lock. Since any Java object can act as lock, the wait and notify methods are the Object class' methods.

In [ ]:
class Data {
    private String data;
    private Object lock = new Object();
    private boolean sent = true;

    public void send(String data) {
        synchronized (lock) {
            while (!sent) {
                try {
                    lock.wait();
                } catch (InterruptedException e) {
                    e.printStackTrace();
                }
            }

            sent = false;
            this.data = data;

            lock.notifyAll();
        }
    }

    public String receive(){
        synchronized (lock) {
            while (sent) {
                try {
                    lock.wait();
                } catch (InterruptedException e) {
                    e.printStackTrace();
                }
            }

            sent = true;
            lock.notifyAll();

            return this.data;
        }
    }
}

Wait/notify forms the basis of producer consumer pattern.

In [ ]:
class Common {
    private int number;
    private boolean writeable = true;

    public synchronized void setNumber(int number) {
        // If writeable is false, this means the consumer
        // has not consumed the data, wait
        while (!writeable) {
            try {
                wait();
            } catch (InterruptedException e) {
                e.printStackTrace();
            }
        }

        writeable = false;
        this.number = number;
        System.out.println("Number " + this.number + " set");

        notify(); // If we miss this all consumers will continue
                  // waiting. Here we notify all waiting consumers that
                  // that new data is ready
    }

    public synchronized int getNumber() {
        // Producer has not produced new data, so better wait
        while (writeable) {
            try {
                wait();
            } catch (InterruptedException e) {
                e.printStackTrace();
            }
        }

        writeable = true;
        notify(); // Notify producer that data has been consumed
                  // they can stop waiting (after this method
                  // has been executed)

        System.out.println("Number " + this.number + " read");
        return this.number;
    }
}

class Producer extends Thread {
    private Common common;

    public Producer(Common common) {
        this.common = common;
    }

    @Override
    public void run() {
        // Produce even numbers sequentially
        for (int i = 0; i < 100; i++) {
            if (i % 2 == 0) {
                common.setNumber(i);
            }
        }
    }
}

class Consumer extends Thread {
    private Common common;

    public Consumer(Common common) {
        this.common = common;
    }

    @Override
    public void run() {
        // Consume all generated numbers
        for (int i = 0; i < 50; i++) {
            common.getNumber();
        }
    }
}

What if a waiting thread is never notified. There is another overloaded version `void wait(long timeout)` which waits only for a certain duration. It is important to note that the wait method is called from inside a while loop and not an if block. Why?:
1. Thread A calls a method that acquires the synchronization lock.
2. Thread A examines a state flag and determines that the data is not in the desired state.
3. Thread A calls the wait() method, which frees the lock.
4. Thread B calls a method that acquires the same synchronization lock.
5. Thread C calls a method that blocks waiting for the lock.
6. Thread B sets the state flag and calls the notifyAll() method.
7. Thread B finishes its method and frees the lock.
8. Thread C acquires the lock and proceeds to process the data; it sees that the data is in the desired state, so it processes the data and resets the state flag.
9. Thread C exits without needing to wait.
10. Thread A receives the notification and wakes up.

This above sequence is not the one we expected. Therefore it is necessary to check the condition after waking from wait.

### ThreadLocal
ThreadLocal enables us to create variables that can only be read and written by the same thread. Consider the code below:

In [1]:
class BirthdayService{
    public String getFormattedBirthdate(int userId){
        Date birthDate = getBirthdate(userId);
        return new SimpleDateFormat("yyyy-mm-dd").format(birthDate);
    }
    
    // More methods
}

class Demo{
    public static void main(String[] args){
        BirthdayService service = new BirthdayService();
        
        Thread t1 = new Thread(()->{
            String birthDate = service.getBirthdate(50);
            System.out.println(birthDate);
        });
        
        Thread t2 = new Thread(()->{
            String birthDate = service.getBirthdate(150);
            System.out.println(birthDate);
        });
        
        t1.start();
        t2.start();
    }
}

We created two `SimpleDateFormat` objects, one per each thread call. What about the following modification?

In [ ]:
class Demo{
    public static void main(String[] args){
        BirthdayService service = new BirthdayService();
        
        for(int i=0; i<1000; i++){
            Thread t = new Thread(()->{
                String birthDate = service.getBirthdate(i);
                System.out.println(birthDate);
            });

            t.start();
        }
    }
}

Now we are creating 1000 SimpleDateFormat objects. We are also creating 1000 new threads. As we'll see later, 1000 new threads are not ideal, they are expensive to create and consume memory. We have 1000 tasks, we may want to use only 10 threads to execute these 1000 tasks. We can't use the same shared SimpleFormat object, but we can use the same SimpleFormat per thread. That is where ThreadLocal comes into play. To maintain 10 threads, we can make use of thread pools:

In [ ]:
public class ThreadSafeFormatter {
    // We are extending ThreadLocal because we have to initialize the
    // SimpleDateFormat in a specific way
    public static ThreadLocal<SimpleDateFormat> simpleDateFormat = new ThreadLocal<SimpleDateFormat>() {
        @Override
        protected SimpleDateFormat initialValue() {
            return new SimpleDateFormat("yyyy-mm-dd");
        };
    };
}

class BirthdayService{
    public String getFormattedBirthdate(int userId){
        Date birthDate = getBirthdate(userId);
        return new ThreadSafeFormatter.simpleDateFormat.get().format(birthDate);
    }
    
    // More methods
}

// We'll be using a FixedThreadPool
class Demo{
    public static void main(String[] args){
        BirthdayService service = new BirthdayService();
        
        ExecutorService service = Executors.newFixedThreadPool(10);
        // Each thread pool thread will have the its own
        // SimpleDateFormat object for all the tasks it gets
        for(int i=0; i<1000; i++){
            service.execute(()->{
                String birthDate = service.getBirthdate(i);
                System.out.println(birthDate);
            });
        }
        
        // Waiting for executor to finish
    }
}

If the `get` method is being called for the first time (by a thread), it internally calls the `initialValue` method. Subsequent calls to get by the same thread, returns the same object (with same initialized value).  

An important point to remember is that whenever the object has been completely used by a thread, we need to remove that object from ThreadLocal.

In [ ]:
ThreadSafeFormatter.simpleDateFormat.remove();